In [ ]:
%pylab inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import seaborn as sns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
hmda1 = pd.read_csv('hmda14a.csv')
hmda2 = pd.read_csv('hmda14b.csv')
hmda3 = pd.read_csv('hmda14c.csv')

hmda1.columns

# Remove rows with high null percentage or duplicate information
def cleaned(df):
    # Remove rows with high null percentage or duplicate information
    keepcols = ['RESP_ID', 'AGENCY', 'LOAN_TYPE', 'PROP_TYPE', 'PURPOSE', 'OCCUPANCY', 'AMNT', 'PRE_APP', \
    'ACTION', 'STATE', 'COUNTY', 'APP_ETH', 'COAPP_ETH', 'APP_RACE1', 'COAP_RACE1', \
    'APP_SEX', 'COAP_SEX', 'APP_INC', 'PURCHTYPE','HOEPA', 'LIENSTATUS', \
    'POP','MINPOP',  'MED_INC', 'TCT2MSA', 'OWNOCC', 'OWNOCC4', 'APP_FLAG']
    df = df[keepcols]

    #replace all '\N' with np.nan
    df = df.replace('\\N',np.nan)

    # Replace "\N" Applicant Incomes with median income for their tract
    df.MED_INC = df.MED_INC.astype("float64")/1000
    df.loc[df.APP_INC.isna(), "APP_INC"] = df.MED_INC
    
    df.dropna(inplace=True)

    # Change percentages to multipliers
    df.MINPOP = df.MINPOP.astype("float64")/100
    df.TCT2MSA = df.TCT2MSA.astype("float64")/100

    # Convert to correct formats
    df.OWNOCC = df.OWNOCC.astype("float64")
    df.OWNOCC4 = df.OWNOCC4.astype("float64")
    df.POP = df.POP.astype("float64")
    return df

 keepcols = ['RESP_ID', 'AGENCY', 'LOAN_TYPE', 'PROP_TYPE', 'PURPOSE', 'OCCUPANCY', 'AMNT', 'PRE_APP', \
    'ACTION', 'STATE', 'COUNTY', 'APP_ETH', 'COAPP_ETH', 'APP_RACE1', 'COAP_RACE1', \
    'APP_SEX', 'COAP_SEX', 'APP_INC', 'PURCHTYPE', 'SPREAD', 'HOEPA', 'LIENSTATUS', \
    'POP','MINPOP',  'MED_INC', 'TCT2MSA', 'OWNOCC', 'OWNOCC4', 'APP_FLAG']
hmda1 = hmda1[keepcols]
test = hmda1.replace('\\N', np.nan)

test.isna().sum()

df1 = cleaned(hmda1)

df1.head()

df2 = cleaned(hmda2)

df2.head()

df3 = cleaned(hmda3)

df3.head()

### Starting merge dataset

fips = pd.read_csv("fips.csv", header=None)
msaoff = pd.read_csv("msaoff.csv")
rp = pd.read_csv("rp.csv")
ts = pd.read_csv("ts.csv")

fips.head()

fips.rename(columns={0:"STATE",1:"STATE_ID",2:"COUNTY_ID", 3:"COUNTY", 4:"H-something"},inplace=True)
fips.COUNTY_ID = fips.COUNTY_ID.astype("float64")
df1 = df1.merge(fips, how="left", left_on=["STATE",'COUNTY'], right_on=["STATE_ID",'COUNTY_ID'])
df1.drop(columns=(["STATE_x","COUNTY_x","STATE_ID","COUNTY_ID"]),inplace=True)
df1.rename(columns=({"STATE_y":"STATE","COUNTY_y":"COUNTY"}),inplace=True)

df2 = df2.merge(fips, how="left", left_on=["STATE",'COUNTY'], right_on=["STATE_ID",'COUNTY_ID'])
df2.drop(columns=(["STATE_x","COUNTY_x","STATE_ID","COUNTY_ID"]),inplace=True)
df2.rename(columns=({"STATE_y":"STATE","COUNTY_y":"COUNTY"}),inplace=True)

df3 = df3.merge(fips, how="left", left_on=["STATE",'COUNTY'], right_on=["STATE_ID",'COUNTY_ID'])
df3.drop(columns=(["STATE_x","COUNTY_x","STATE_ID","COUNTY_ID"]),inplace=True)
df3.rename(columns=({"STATE_y":"STATE","COUNTY_y":"COUNTY"}),inplace=True)

ts = ts[["RESP_ID","RESP_NAME","RESP_ADD","RESP_CITY","RESP_STATE","RESP_ZIP","ASSETS","OTHLEND",'AGENCY']]

ts['RESP_ID'] = ts.RESP_ID.str.lstrip('0')
ts['RESP_ID'] = ts.RESP_ID.astype('str')
df1['RESP_ID'] = df1.RESP_ID.astype('str')
df1['RESP_ID'] = df1.RESP_ID.str.lstrip('0')
merged1 = df1.merge(ts, how="left", left_on=['RESP_ID','AGENCY'], right_on=['RESP_ID','AGENCY'])

df2['RESP_ID'] = df2.RESP_ID.astype('str')
df2['RESP_ID'] = df2.RESP_ID.str.lstrip('0')
merged2 = df2.merge(ts, how="left", left_on=['RESP_ID','AGENCY'], right_on=['RESP_ID','AGENCY'])

df3['RESP_ID'] = df3.RESP_ID.astype('str')
df3['RESP_ID'] = df3.RESP_ID.str.lstrip('0')
merged3 = df3.merge(ts, how="left", left_on=['RESP_ID','AGENCY'], right_on=['RESP_ID','AGENCY'])

merged = pd.concat([merged1,merged2,merged3],ignore_index=True)

merged1 = merged[(merged.ACTION != 4) & (merged.ACTION != 5) & (merged.ACTION != 7) & (merged.ACTION != 8) ]

merged1['APPROVED'] = merged1.ACTION.apply(lambda x: 1 if (x==1) | (x==2) | (x==6) else 0)

merged1.shape

merged1.APPROVED.value_counts()

merged1.ACTION.value_counts()

merged1.columns

merged1 = merged1[merged1.COAPP_ETH != 4]

merged1 = merged1[(merged1.COAP_RACE1 != 6) & (merged1.COAP_RACE1 != 7)]

merged1 = merged1[(merged1.COAP_SEX != 3) & (merged1.COAP_SEX != 4)]

merged2 = merged1.drop(columns = ['ACTION','RESP_ID','RESP_NAME',"RESP_ADD","RESP_CITY","RESP_STATE",\
                   "RESP_ZIP","APP_FLAG",'H-something'])

merged2 = merged2[(merged2.APP_SEX==1)|(merged2.APP_SEX==2)]

merged2 = merged2[(merged1.APP_RACE1!=7)&(merged2.APP_RACE1!=6)]

merged2 = merged2[(merged1.APP_ETH==1)|(merged2.APP_ETH==2)]

merged2.shape

sb = merged2.sample(frac=0.10,random_state=0)

sb.shape

sb.to_csv('subset1.csv')

sb.head()

sb2 = sb.drop(columns=['COUNTY'])

sb3 = pd.get_dummies(sb2, columns=['AGENCY','LOAN_TYPE','PROP_TYPE','PURPOSE','OCCUPANCY','PRE_APP','APP_ETH',\
                            'COAPP_ETH','APP_RACE1','COAP_RACE1','APP_SEX','COAP_SEX','PURCHTYPE','HOEPA',\
                            'LIENSTATUS','STATE','OTHLEND'])

sb3.shape

sb3.head()

sb3.dtypes

sb3['AMNT'] = sb3.AMNT.astype('float64') 
sb3['APP_INC'] = sb3.APP_INC.astype('float64')
sb3['ASSETS'] = sb3.ASSETS.astype('float64')
sb3['APPROVED'] = sb3.APPROVED.astype('uint8')

df_scale = sb3.copy()

from sklearn.preprocessing import StandardScaler
df_scale = sb3.select_dtypes(include='float64')
df_not_scale = sb3.drop(columns=df_scale.columns)

scaler = StandardScaler()
df_norm = scaler.fit_transform(df_scale)
df_norm = pd.DataFrame(df_norm, columns=df_scale.columns, index=df_scale.index)
sb4 = pd.concat([df_norm, df_not_scale], axis=1)

sb4.head()

sb4.to_csv('before_ml.csv')